In [1]:
%status

{'status': 'healthy',
 'startTime': 'Fri Aug 16 00:24:27 UTC 2024',
 'dbEngineVersion': '1.3.2.1.R1',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.7.1'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'SlowQueryLogs': 'disabled',
  'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'StrictTimeoutValidation': 'false',
  'clusterQueryTimeoutInMs': '120000',
  'SlowQueryLogsThreshold': '5000'}}

In [2]:
%%oc
MATCH (n)
DETACH DELETE n

In [4]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

In [5]:
%%oc
MATCH (n)
RETURN COUNT(n)

Execute DISTINCT query

In [16]:
%%oc
MATCH (e:Entity) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
RETURN DISTINCT(e.service_provider)

Execute insertion of node including gFD verification (for existing equivalence class), Amazon Neptune does not support CALL procedures

In [39]:
%%oc
MERGE (new:Entity{name: 'new', service_provider: 'Mossack Fonseca', sourceID: 'Panama Papers', valid_until: 'The Panama Papers data is current through 2015'})
WITH new AS newnode
MATCH (e:Entity) WHERE
e.service_provider IS NOT NULL AND
e.sourceID IS NOT NULL AND
e.valid_until IS NOT NULL
WITH e.service_provider AS provider, COUNT(DISTINCT(e.sourceID + e.valid_until)) AS amount
WHERE amount > 1
RETURN provider, amount

Delete new node (return count to see that node deleted, not measuring time for this operation)

In [40]:
%%oc
MATCH (e:Entity) WHERE e.name = 'new' DELETE e
RETURN COUNT(e)

Execute insertion of node including gFD verification (including new equivalence class), Amazon Neptune does not support CALL procedures

In [64]:
%%oc
MERGE (new:Entity{name: 'new', service_provider: 'new', sourceID: 'new', valid_until: 'new'})
WITH new AS newnode
MATCH (e:Entity) WHERE
e.service_provider IS NOT NULL AND
e.sourceID IS NOT NULL AND
e.valid_until IS NOT NULL
WITH e.service_provider AS provider, COUNT(DISTINCT(e.sourceID + e.valid_until)) AS amount
WHERE amount > 1
RETURN provider, amount

Delete new node (return count to see that node deleted, not measuring time for this operation)

In [65]:
%%oc
MATCH (e:Entity) WHERE e.name = 'new' DELETE e
RETURN COUNT(e)

Normalize graph: Create Provider nodes for each equivalence class

In [1]:
%%oc
MATCH (e:Entity) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until)
WITH DISTINCT e.service_provider AS provider, e.sourceID AS ids, e.valid_until AS valid
CREATE (p:Provider{service_provider: provider, sourceID: ids, valid_until: valid})

Normalize graph: create edges and remove properties related to equivalence class from original nodes

In [2]:
%%oc
MATCH (e:Entity),(p:Provider) WHERE
EXISTS(e.service_provider) AND EXISTS(e.sourceID) AND EXISTS(e.valid_until) AND
e.service_provider = p.service_provider
CREATE (e)<-[:PROVIDED_TO]-(p)

In [7]:
%%oc
MATCH (e:Entity)
REMOVE e.service_provider, e.sourceID, e.valid_until

Execute in normalised graph

Execute DISTINCT query

In [28]:
%%oc
MATCH (p:Provider) WHERE
EXISTS(p.service_provider) AND EXISTS(p.sourceID) AND EXISTS(p.valid_until)
RETURN DISTINCT(p.service_provider)

Execute insertion of node (for existing equivalence class)

In [30]:
%%oc
MATCH (p:Provider{service_provider: 'Mossack Fonseca'})
MERGE (e:Entity{name: 'new'})<-[r:PROVIDED_TO]-(p)

Delete new node(s) and edges if created (return count to see that delete was succesful, not measuring time for this operation)

In [32]:
%%oc
MATCH (e:Entity{name: 'new'})
OPTIONAL MATCH (p:Provider{service_provider: 'new'})
DETACH DELETE e,p
RETURN COUNT(e), COUNT(p)

Execute insertion of node including (including new equivalence class node)

In [58]:
%%oc
MERGE (e:Entity{name: 'new'})<-[r:PROVIDED_TO]-(p:Provider{service_provider: 'new', sourceID: 'new', valid_until: 'new'})

Delete new node(s) and edges if created (return count to see that delete was succesful, not measuring time for this operation)

In [59]:
%%oc
MATCH (e:Entity{name: 'new'})
OPTIONAL MATCH (p:Provider{service_provider: 'new'})
DETACH DELETE e,p
RETURN COUNT(e), COUNT(p)